<a href="https://colab.research.google.com/github/HanhengHe/BigDataCourseProj/blob/main/Workspace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [74]:
class Generator(nn.Module):
    def __init__(self, noiseSize=200, ct_deepth=128):
        super(Generator, self).__init__()
        
        self.model_noise_channel = nn.Sequential(
            nn.ConvTranspose2d(noiseSize, ct_deepth * 4, 12, 1, 0),
            nn.BatchNorm2d(ct_deepth * 4, 0.2),
            nn.LeakyReLU()
            )
        
        self.model_label_channel = nn.Sequential(
            nn.ConvTranspose2d(50, ct_deepth * 4, 12, 1, 0),
            nn.BatchNorm2d(ct_deepth * 4, 0.2),
            nn.LeakyReLU()
            )

        self.model_kernal = nn.Sequential(
            nn.ConvTranspose2d(ct_deepth * 8, ct_deepth * 4, 6, 2, 1),
            nn.BatchNorm2d(ct_deepth * 4, 0.2),
            nn.LeakyReLU(),
            nn.ConvTranspose2d(ct_deepth * 4, ct_deepth * 2, 6, 2, 1),
            nn.BatchNorm2d(ct_deepth * 2, 0.2),
            nn.LeakyReLU(),
            nn.ConvTranspose2d(ct_deepth * 2, ct_deepth, 6, 2, 1),
            nn.BatchNorm2d(ct_deepth, 0.2),
            nn.LeakyReLU(),
            nn.ConvTranspose2d(ct_deepth, 3, 8, 2, 1),
            nn.Tanh()
            )

    def forward(self, x, label):
        x_0 = self.model_noise_channel(x)
        x_1 = self.model_label_channel(label)
        x = torch.cat([x_0, x_1], 1)
        x = self.model_kernal(x)
        return x


class Discriminator(nn.Module):
    def __init__(self, ct_deepth=128):
        super(Discriminator, self).__init__()

        self.model_noise_channel = nn.Sequential(
            nn.ConvTranspose2d(3, ct_deepth / 2, 4, 2, 1),
            nn.BatchNorm2d(ct_deepth / 2, 0.2),
            nn.LeakyReLU()
            )
        
        self.model_label_channel = nn.Sequential(
            nn.ConvTranspose2d(50, ct_deepth / 2, 4, 2, 1),
            nn.BatchNorm2d(ct_deepth / 2, 0.2),
            nn.LeakyReLU()
            )

        self.model_kernal = nn.Sequential(
            nn.ConvTranspose2d(ct_deepth, ct_deepth * 2, 4, 2, 1),
            nn.BatchNorm2d(ct_deepth * 2, 0.2),
            nn.LeakyReLU(),
            nn.ConvTranspose2d(ct_deepth * 2, ct_deepth * 4, 4, 2, 1),
            nn.BatchNorm2d(ct_deepth * 4, 0.2),
            nn.LeakyReLU(),
            nn.ConvTranspose2d(ct_deepth * 4, ct_deepth * 8, 4, 2, 1),
            nn.BatchNorm2d(ct_deepth * 8, 0.2),
            nn.LeakyReLU(),
            nn.ConvTranspose2d(ct_deepth * 8, 1, 4, 1, 0),
            nn.Sigmoid()
            )

    def forward(self, x, label):
        x_0 = self.model_noise_channel(x)
        x_1 = self.model_label_channel(label)
        x = torch.cat([x_0, x_1], 1)
        x = self.model_kernal(x)
        return x

In [46]:
CPU = torch.device("cpu")
DEVICE = torch.device("cuda:0") if torch.cuda.is_available() else CPU
# DEVICE = CPU
print("DEVICE = %s" % (DEVICE))

DEVICE = cpu


In [75]:
G = Generator()
y = G(torch.randn((1, 200, 1, 1)), torch.randn((1, 50, 1, 1)))
print(y.shape)

torch.Size([1, 3, 224, 224])
